In [1]:
import pandas as pd

In [2]:
titanic = pd.read_csv(r'C:/Users/carol/Downloads/train2.csv')

In [3]:
titanic.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titulos
0,0,3,male,22.0,1,0,7.2500,S,Mr
1,1,1,female,38.0,1,0,71.2833,C,Mrs


##### com a árvore de decisão iremos obter um erro ao tentar transformar as colunas - 'could not convert string to float'

##### a maior parte dos modelos não irão conseguir receber valores de texto e trabalhar com eles, sendo assim, devemos transformar os valores em númericos

In [4]:
titanic.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

###### temos apenas dois valores, então podemos considerar 0 ou 1

In [5]:
titanic['Is_Male'] = titanic.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [6]:
titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titulos,Is_Male
0,0,3,male,22.0,1,0,7.2500,S,Mr,1
1,1,1,female,38.0,1,0,71.2833,C,Mrs,0
2,1,3,female,26.0,0,0,7.9250,S,Miss,0
3,1,1,female,35.0,1,0,53.1000,S,Mrs,0
4,0,3,male,35.0,0,0,8.0500,S,Mr,1


In [7]:
titanic[['Sex','Is_Male']].value_counts()

Sex     Is_Male
male    1          577
female  0          314
dtype: int64

In [8]:
titanic = titanic.drop('Sex',axis = 1)

In [9]:
#importando o modelo
from sklearn.tree import DecisionTreeClassifier

#criando o classificador
clf = DecisionTreeClassifier(random_state=0)

#separando em x e y

X = titanic.drop('Survived',axis = 1)
y = titanic.Survived

#fazendo o fit
clf = clf.fit(X, y)

ValueError: could not convert string to float: 'S'

#### Mesmo problema em coluna diferente

In [10]:
titanic.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [12]:
# vamos resolver o problema através do One Hot Encoder do sklearn

In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
ohe = OneHotEncoder()

In [15]:
ohe = ohe.fit(X[['Embarked']])

In [16]:
ohe.transform(X[['Embarked']]).toarray()

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [17]:
ohe_df = pd.DataFrame(ohe.transform(X[['Embarked']]).toarray())

In [18]:
ohe_df.head(2)

,0,1,2
0,0.0,0.0,1.0
1,1.0,0.0,0.0


In [22]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [23]:
ohe.get_feature_names_out()

array(['Embarked_C', 'Embarked_Q', 'Embarked_S'], dtype=object)

In [19]:
ohe_df.columns = ohe.get_feature_names_out()

In [20]:
ohe_df.head(3)

,Embarked_C,Embarked_Q,Embarked_S
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0


In [21]:
titanic.head(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Titulos,Is_Male
0,0,3,22.0,1,0,7.2500,S,Mr,1
1,1,1,38.0,1,0,71.2833,C,Mrs,0
2,1,3,26.0,0,0,7.9250,S,Miss,0


In [24]:
titanic = pd.concat([titanic,ohe_df],axis = 1)

In [26]:
titanic.head(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Titulos,Is_Male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,S,Mr,1,0.0,0.0,1.0
1,1,1,38.0,1,0,71.2833,C,Mrs,0,1.0,0.0,0.0
2,1,3,26.0,0,0,7.9250,S,Miss,0,0.0,0.0,1.0


In [27]:
titanic = titanic.drop(['Embarked','Titulos'],axis = 1)

In [28]:
titanic.head(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Is_Male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,1,0.0,0.0,1.0
1,1,1,38.0,1,0,71.2833,0,1.0,0.0,0.0
2,1,3,26.0,0,0,7.9250,0,0.0,0.0,1.0


In [30]:
# agora podemos separar em x e y

X = titanic.drop('Survived', axis = 1)
y = titanic.Survived

In [31]:
#fazendo o fit
clf = clf.fit(X,y)